In [ ]:
pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 21.6 MB/s eta 0:00:00


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.4 MB/s eta 0:00:00


In [ ]:
# Import các thư viện cần thiết
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy
import re
import underthesea # Thư viện tách từ

from sklearn.model_selection import train_test_split # Thư viện chia tách dữ liệu

from transformers import AutoModel, AutoTokenizer # Thư viện BERT


from joblib import dump


In [ ]:
# Hàm load model BERT
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer

In [ ]:
# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip().lower()
    return row

In [ ]:
# Hàm load danh sách các từ vô nghĩa: lắm, ạ, à, bị, vì..
def load_stopwords():
    sw = []
    with open("stopword.txt", encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        sw.append(line.replace("\n",""))
    return sw

In [ ]:
import pandas as pd
post_df_full = pd.read_csv("/content/cmtfull.csv")
post_df_full.head(5)

,comment,classify
0,Kiều Linh thôi xong,2.0
1,"E dung tin lung tung, sach hay nhe, ko thich h...",2.0
2,Nguyễn Mạnh Dũng thằng óc cho mua sách củ lìn,1.0
3,Minh dm comment hỗn chiến VKN >< Khôn Lường,1.0
4,Trần Hân Thánh Thần có phải thầy hợp tác vs vợ...,2.0


In [ ]:
def load_data():
    v_text = []
    v_label = []

    dfs = post_df_full

    for d in dfs.comment:
        v_text.append(d)

    i = 0;
    for l in dfs.classify:
        v_label.append(l)

    print(v_label)
    return v_text, v_label

In [ ]:
import torch
import numpy as np
import underthesea

def make_bert_features(v_text):
    global phobert, sw, tokenizer
    v_tokenized = []
    max_len = 100  # Mỗi câu dài tối đa 100 từ
    for i_text in v_text:
        print("Đang xử lý line =", i_text)
        # Phân thành từng từ
        line = underthesea.word_tokenize(i_text)
        # Lọc các từ vô nghĩa
        filtered_sentence = [w for w in line if not w in sw]
        # Ghép lại thành câu như cũ sau khi lọc
        line = " ".join(filtered_sentence)
        line = underthesea.word_tokenize(line, format="text")
        # print("Word segment  =", line)
        # Tokenize bởi BERT
        line = tokenizer.encode(line)
        v_tokenized.append(line)

    # Chèn thêm số 1 vào cuối câu nếu như không đủ 100 từ
    padded = np.zeros((len(v_tokenized), max_len), dtype=np.int64)
    for i, line in enumerate(v_tokenized):
        padded[i, :len(line)] = line[:max_len]

    attention_mask = np.where(padded != 0, 1, 0)
    
    # Chuyển thành tensor
    padded = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)

    # Lấy features dầu ra từ BERT
    with torch.no_grad():
        last_hidden_states = phobert(input_ids=padded, attention_mask=attention_mask)

    v_features = last_hidden_states[0][:, 0, :].numpy()
    print(v_features.shape)
    return v_features


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...")
sw = load_stopwords()
print("Đã nạp xong danh sách các từ vô nghĩa")

print("Chuẩn bị nạp model BERT....")
phobert, tokenizer = load_bert()
print("Đã nạp xong model BERT.")


Chuẩn bị nạp danh sách các từ vô nghĩa (stopwords)...
Đã nạp xong danh sách các từ vô nghĩa
Chuẩn bị nạp model BERT....


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Đã nạp xong model BERT.


In [ ]:
print("Chuẩn bị load dữ liệu....")
text, label = load_data()
print("Đã load dữ liệu xong")

Chuẩn bị load dữ liệu....
[2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 0.0, 2.0, 1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 2.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0, 2.0, 2.0, 0.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 2.0, 1.0, 2.0, 1.0, 0.0, 1.0, 0.0, 1.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 0.0, 2.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 0.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 0.0, 2.0, 2.0, 2.0, 1.0, 2.0, 0.0, 2.0, 2.0, 1.0, 2.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0

In [ ]:
print("Chuẩn bị tạo features từ BERT.....")
features = make_bert_features(text)
print("Đã tạo xong features từ BERT")

Chuẩn bị tạo features từ BERT.....
Đang xử lý line = Kiều Linh  thôi xong
Đang xử lý line = E dung tin lung tung, sach hay nhe, ko thich hoc thi thoi.thay giang vien dai hoc sp Ha Noi
Đang xử lý line = Nguyễn Mạnh Dũng thằng óc cho mua sách củ lìn
Đang xử lý line = Minh dm comment hỗn chiến VKN >< Khôn Lường
Đang xử lý line = Trần Hân Thánh Thần có phải thầy hợp tác vs vợ thầy H ko cm 😂😂
Đang xử lý line = Khả Dung mới nói xong
Đang xử lý line = Phan Thị Thảo Quyên m muốn đeo theo ông này thì hãy suy nghĩ, t đã xem sách ổng lụ đạn, dở dã man mà đắt kinh khủng
Đang xử lý line = Võ HTr Bạn có làm hết bài tập trong đó chưa hyhy :3
Đang xử lý line = Phạm Hoàng Hải :3 không biết thì vô đây đọc nè a :3
Đang xử lý line = Thúy Thanh mày à tao vừa định học :v mày nhắc rồi thêm này nữa
Đang xử lý line = đề nghị admin tát vào dái 10 cái vì tội vu khống.tôi đã tìm ra thí sinh đó ở cụm thi và đạt tròn 10 hóa.
Đang xử lý line = Thiện Minh Phạm Quốc Toản hồi trước có học thầy này không
Đang xử lý line

Token indices sequence length is longer than the specified maximum sequence length for this model (329 > 256). Running this sequence through the model will result in indexing errors


Đang xử lý line = T từng mất gốc toàn bộ kiến thức từ nửa đầu c3 đổ xuống. T tự ôn hết toán lý hóa sinh dưới sự định hướng của thầy cô chỉ trong nửa cuối năm lớp 11 và năm 12, đậu 2 trường ĐH khối A và B thuộc top. Cho nên là lão ấy có PR thế PR nữa thì tất cả vẫn chỉ là tự bản thân thôi.
Đang xử lý line = Hoang Kim sách bà mua của thầy này nè hoàng
Đang xử lý line = Ko chỉ có 99 bị thầy hấp dẫn bằng ngôn ngữ cường điệu đâu,ngay cả mình khóa của mình 95-ers cũng bị,lúc ấy mình cứ tưởng thật học theo gần chết,sau này sắp ra trường  đi dạy gia sư cho mấy e cấp3 thì thấy công thức nhanh này nọ cũng chẳng gì là thần thánh cả nếu mình nắm vững căn bản
Đang xử lý line = mình học không có cuốn sách nào.. vẫn sống sót qua kì thi.. kết quả thế nào phụ thuộc nhiều yếu tố.. mua sách không ngó ngàng cũng coi như bỏ đi.. khả năng tập trung không có cũng không học hành nghiêm túc lâu được.. còn nữa.. ai sống sao người đó tự biết.. muốn tìm hiểu thì hỏi người đi trước.. hỏi cái người mua sách về đọc 

In [ ]:
# Tìm các phần tử bị NaN
phantu_nan = np.isnan(label)

label = np.nan_to_num(label, nan=2.0)

In [ ]:
# Phân chia dữ liệu train, test
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=48)

print("Chuẩn bị train model NB....")
from sklearn.naive_bayes import GaussianNB
cl = GaussianNB()
cl.fit(features, label)

sc = cl.score(X_test, y_test)
print('Kết quả train model, độ chính xác = ', sc*100, '%')

# Save model
dump(cl, 'save_model.pkl')
print("Đã lưu model NB vào file save_model.pkl")

Chuẩn bị train model NB....
Kết quả train model, độ chính xác =  60.71428571428571 %
Đã lưu model NB vào file save_model.pkl
